<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Building The Prediction Model

### Riccardo Spolaor (864877)

</center>

Modifico il dataframe in modo da poter ottenere un modello adatto alla predizione:
* Rinomino ***Winner*** in ***PlayerA*** e ***Loser*** in ***PlayerB***;
* Rinomino tutte le colonne con informazioni legate al vincitore e al perdente in modo da riflettere il legame ai nuovi nomi assegnati;
* Creo la colonna ***Winner*** che assume i valori:
    * 0 se ha vinto ***PlayerA***;
    * 1 se ha vinto ***PlayerB***;
* Assegno a ***Winner*** tutti valori 0;
* Duplico il dataframe e faccio lo swapping tra tutte le colonne con informazioni sul Giocatore A e quelle con informazioni sul Giocatore B;
* Assegno il valore 1 a ogni entry della colonna ***Winner*** del nuovo dataframe;
* Unisco i due dataframe.

In [1]:
import pandas as pd

In [2]:
dataframe = pd.read_csv('./datasets/dataframe02.csv', low_memory = False)

In [3]:
dataframe.columns


Index(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court',
       'Round', 'Best of', 'Winner',
       ...
       'Isner J.A', 'Isner J.B', 'Moya C.A', 'Moya C.B', 'Davydenko N.A',
       'Davydenko N.B', 'Gonzalez F.A', 'Gonzalez F.B', 'OtherA', 'OtherB'],
      dtype='object', length=101)

In [4]:
def renameColumns():
    dataframe.rename(columns={'Winner':'PlayerA', 
                          'Loser':'PlayerB',
                          'WRank':'RankA',
                          'WRankAdded':'RankFilledA',
                          'LRank':'RankB',
                          'LRankAdded':'RankFilledB',
                          'W1':'1A',
                          'L1':'1B',
                          'W2':'2A',
                          'L2':'2B',
                          'W3':'3A',
                          'L3':'3B',
                          'W4':'4A',
                          'L4':'4B',
                          'W5':'5A',
                          'L5':'5B',
                          'Wsets':'setsA',
                          'Lsets':'setsB',
                          'WPts':'PtsA',
                          'LPts':'PtsB',
                          'WPtsAdded':'PtsFilledA',
                          'LPtsAdded':'PtsFilledB',
                          'MaxW':'MaxA',
                          'MaxWAdded':'MaxFilledA',
                          'MaxL':'MaxB',
                          'MaxLAdded':'MaxFilledB',
                          'AvgW':'AvgA',
                          'AvgWAdded':'AvgFilledA',
                          'AvgL':'AvgB',
                          'AvgLAdded':'AvgFilledB',
                         }, inplace = True)
    dataframe.insert(0, column ='Winner', value = 0)

In [5]:
renameColumns()
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Round', 'Best of',
       ...
       'Isner J.A', 'Isner J.B', 'Moya C.A', 'Moya C.B', 'Davydenko N.A',
       'Davydenko N.B', 'Gonzalez F.A', 'Gonzalez F.B', 'OtherA', 'OtherB'],
      dtype='object', length=102)

In [6]:
def getInvertedDataFrame(colToInvert):
    
    players = dataframe['PlayerA'].value_counts().subtract(
        dataframe['PlayerB'].value_counts(), fill_value = 0).sort_values(ascending = False).index.tolist()[:25]
    
    colToInvert += players
    
    dataframeInverted = dataframe.copy(deep = True)

    dataframeInverted['Winner'] = 1
    
    for f in colToInvert:
        dataframeInverted[[f+'A', f+'B']] = dataframeInverted[[f+'B', f+'A']]
    
    return dataframeInverted

In [7]:
dataframeInverted = getInvertedDataFrame(['Player', 'Rank', 'RankFilled', 'Pts', 'PtsFilled', '1', 
                   '2', '3', '4', '5', 'sets', 'Max', 'Avg', 'Other', 'MaxFilled', 'AvgFilled'])

In [8]:
dataframeInverted.head()

,Winner,csvID,ATP,Location,Tournament,Date,Series,Court,Round,Best of,...,Isner J.A,Isner J.B,Moya C.A,Moya C.B,Davydenko N.A,Davydenko N.B,Gonzalez F.A,Gonzalez F.B,OtherA,OtherB
0,1,0,1.0,2,0,2001-01-01,0,1,0,3.0,...,0,0,0,0,0,0,0,0,1,1
1,1,0,1.0,2,0,2001-01-01,0,1,0,3.0,...,0,0,0,0,0,0,0,0,1,1
2,1,0,1.0,2,0,2001-01-01,0,1,0,3.0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,1.0,2,0,2001-01-01,0,1,0,3.0,...,0,0,0,0,0,0,0,0,1,1
4,1,0,1.0,2,0,2001-01-01,0,1,0,3.0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
dataframe = dataframe.append(dataframeInverted, sort = False, ignore_index = False)

In [10]:
def sortMatchesByDate ():
    dataframe['Index'] = dataframe.index
    dataframe.sort_values(['csvID', 'ATP','Index'],inplace = True)
    dataframe.drop('Index', axis=1, inplace = True)
    
sortMatchesByDate()

In [11]:
dataframe.columns

Index(['Winner', 'csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series',
       'Court', 'Round', 'Best of',
       ...
       'Isner J.A', 'Isner J.B', 'Moya C.A', 'Moya C.B', 'Davydenko N.A',
       'Davydenko N.B', 'Gonzalez F.A', 'Gonzalez F.B', 'OtherA', 'OtherB'],
      dtype='object', length=102)

In [12]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head(50)[['PlayerA', 'PlayerB','MaxA', 'MaxB', 'AvgA', 'AvgB', 'MaxFilledA', 'MaxFilledB', 'AvgFilledA',
                   'AvgFilledB']]

,PlayerA,PlayerB,MaxA,MaxB,AvgA,AvgB,MaxFilledA,MaxFilledB,AvgFilledA,AvgFilledB
0,Clement A.,Gaudenzi A.,1.953528,5.753002,1.816889,3.339702,1,1,1,1
1,Gaudenzi A.,Clement A.,5.753002,1.953528,3.339702,1.816889,1,1,1,1
2,Goldstein P.,Jones A.,1.953528,5.753002,1.816889,3.339702,1,1,1,1
3,Jones A.,Goldstein P.,5.753002,1.953528,3.339702,1.816889,1,1,1,1
4,Haas T.,Smith L.,1.120000,4.000000,1.110000,3.750000,0,0,0,0
5,Smith L.,Haas T.,4.000000,1.120000,3.750000,1.110000,0,0,0,0
6,Henman T.,Rusedski G.,1.953528,5.753002,1.816889,3.339702,1,1,1,1
7,Rusedski G.,Henman T.,5.753002,1.953528,3.339702,1.816889,1,1,1,1
8,Hewitt L.,Arthurs W.,1.280000,3.800000,1.200000,3.550000,0,0,0,0
9,Arthurs W.,Hewitt L.,3.800000,1.280000,3.550000,1.200000,0,0,0,0


In [13]:
dataframe.drop(['Date', 'PlayerA', 'PlayerB', '1A', '1B', '2A', '2B', '3A', '3B', '4A', 
                '4B', '5A', '5B', 'setsA', 'setsB', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover'], axis = 1, inplace = True)

In [14]:
dataframe = dataframe.reset_index(drop = True)
dataframe.head()

,Winner,csvID,ATP,Location,Tournament,Series,Court,Round,Best of,RankA,...,Isner J.A,Isner J.B,Moya C.A,Moya C.B,Davydenko N.A,Davydenko N.B,Gonzalez F.A,Gonzalez F.B,OtherA,OtherB
0,0,0,1.0,2,0,0,1,0,3.0,18.0,...,0,0,0,0,0,0,0,0,1,1
1,1,0,1.0,2,0,0,1,0,3.0,101.0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,1.0,2,0,0,1,0,3.0,81.0,...,0,0,0,0,0,0,0,0,1,1
3,1,0,1.0,2,0,0,1,0,3.0,442.0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,1.0,2,0,0,1,0,3.0,23.0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
dataframe.to_csv('./datasets/dataframe03.csv', index = False)